In [1]:
import os
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
from Ctorch import *
from functions import *
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy.signal import fftconvolve
from skimage import io
from scipy.fftpack import idct
#from model import *
from tqdm import tnrange,tqdm_notebook
import PIL
from dataloaders import *
from visualise import *
from time import time
torch.cuda.set_device(0)
import shutil
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from training import *
from WienerCNN import *
%matplotlib inline

# Training the trainable Wiener Filter

In [2]:
net = WienerDeconvolution(n_filters=16,filter_size=3,grayscale=False);
net.cuda();

In [ ]:
train_wiener(net,100,'dataset/',48,gpu=0,dictname='WNet_color_16x3x3_noise3_lr=0.01_b48');